In [ ]:
import pandas as pd
import requests
import json
from pprint import pprint
# Retrieve Google API key from config.py
from config import gkey

In [ ]:
data_one_path = 'Resources/airbnbmark1.csv'
data_two_path = 'Resources/AB_NYC_2019.csv'
data_one = pd.read_csv(data_one_path)
data_two = pd.read_csv(data_two_path)

In [ ]:
data_one.head(1)

In [ ]:
ident = data_one['id']
name = data_one['name']
rpm = data_one['reviews_per_month']
peoples = data_one['accommodates']
cleaning = data_one['cleaning_fee']
avg_rating = data_one['review_scores_rating']
accuracy = data_one['review_scores_rating']
cleanliness = data_one['review_scores_rating']
checkin = data_one['review_scores_checkin']
comm = data_one['review_scores_communication']
location = data_one['review_scores_location']
value = data_one['review_scores_value']

basic_info_df = pd.DataFrame({'ID': ident,
                             'Room Name': name,
                              'Accomodates': peoples,
                              'Cleaning Fee': cleaning,
                             'Reviews Per Month': rpm,
                             'Rating Score': avg_rating,
                             'Accuracy Score': accuracy,
                             'Cleanliness': cleanliness,
                             'Checkin Score': checkin,
                             'Communication Score': comm,
                             'Location Score': location,
                             'Value Score': value})
basic_info_df.head(2)

In [ ]:
ident2 = data_two['id']
host_id = data_two['host_id']
borough = data_two['neighbourhood_group']
neighborhood = data_two['neighbourhood']
lat = data_two['latitude']
lng = data_two['longitude']
room_type = data_two['room_type']
price = data_two['price']
review_num = data_two['number_of_reviews']
avail = data_two['availability_365']

In [ ]:
basic_two_df = pd.DataFrame({'ID': ident2,
                            'Host ID': host_id,
                            'Borough': borough,
                            'Neighbourhood': neighborhood,
                            'Latitude': lat,
                            'Longitude': lng,
                            'Type Room': room_type,
                            'Price (per night)': price,
                            'Number of Reviews': review_num,
                            'Availability per 365': avail})
basic_two_df

In [ ]:
combined_data = pd.merge(basic_info_df, basic_two_df, on = 'ID')
cleaned_data = combined_data.dropna(how = 'any')
cleaned_data

In [ ]:
# cleaned_data.to_csv('Resources/cleaned_data.csv')

In [ ]:
#Find location for Manhattan, Brooklyn, and Bronx
boroughs_df = pd.read_csv("Resources/Boroughs.csv")
boroughs_df.head()


In [ ]:
boroughs_df["borough lng"]= ""
boroughs_df["borough lat"]= ""
boroughs_df["attraction name"]= ""
boroughs_df["attraction lng"]= ""
boroughs_df["attraction lat"]= ""
boroughs_df

In [ ]:
params = {"key": gkey}

for index, row in boroughs_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['Boroughs']
    state = 'New York'

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    boroughs_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
#     print(cities_lat_lng.url)
    
#     # convert to json
    boroughs_lat_lng = boroughs_lat_lng.json()

    boroughs_df.loc[index, "borough lat"] = boroughs_lat_lng["results"][0]["geometry"]["location"]["lat"]
    boroughs_df.loc[index, "borough lng"] = boroughs_lat_lng["results"][0]["geometry"]["location"]["lng"]

# # Visualize to confirm lat lng appear
boroughs_df.head()

In [ ]:
#Get locations for 1 attraction in each borough
params = {
    "radius": 10000,
    "types": "tourist_attraction",
    "key": gkey
}

for index, row in boroughs_df.iterrows():
    # get lat, lng from df
    lat = row["borough lat"]
    lng = row["borough lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    attraction = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(attraction.url)

    # convert to json
    attraction = attraction.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        boroughs_df.loc[index, "attraction name"] = attraction["results"][0]["name"]
        boroughs_df.loc[index, "attraction lng"] = attraction["results"][0]["geometry"]["location"]["lng"]
        boroughs_df.loc[index, "attraction lat"] = attraction["results"][0]["geometry"]["location"]["lat"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
boroughs_df